In [2]:
# Ignore warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore') 
import pandas as pd
import numpy as np
from sklearn.linear_model import RANSACRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from time import time
import joblib

In [3]:
df = pd.read_excel('/kaggle/input/siklus-menstruasi/dataset_v5.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ClientID                  149 non-null    object 
 1   LengthofCycle             149 non-null    int64  
 2   EstimatedDayofOvulation   149 non-null    float64
 3   LengthofMenses            149 non-null    int64  
 4   Age                       149 non-null    int64  
 5   BMI                       149 non-null    float64
 6   CycleNumber               149 non-null    int64  
 7   LengthofCycle1            149 non-null    int64  
 8   LengthofCycle2            149 non-null    int64  
 9   LengthofCycle3            149 non-null    int64  
 10  LengthOfMenses1           149 non-null    int64  
 11  LengthOfMenses2           149 non-null    int64  
 12  LengthOfMenses3           149 non-null    int64  
 13  MeanOfMenses              149 non-null    float64
 14  EstimatedD

In [4]:
df.head()

,ClientID,LengthofCycle,EstimatedDayofOvulation,LengthofMenses,Age,BMI,CycleNumber,LengthofCycle1,LengthofCycle2,LengthofCycle3,LengthOfMenses1,LengthOfMenses2,LengthOfMenses3,MeanOfMenses,EstimatedDayofOvulation1,EstimatedDayofOvulation2,EstimatedDayofOvulation3
0,nfp8020,28,14.0,5,33,25.790533,4,27,31,27,6,6,7,6.333333,13,17,15
1,nfp8024,26,15.0,5,23,24.958580,4,29,30,28,5,5,5,5.000000,16,18,15
2,nfp8026,27,16.0,6,39,21.454890,4,26,30,25,5,10,5,6.666667,15,19,11
3,nfp8030,25,12.0,6,23,22.462722,4,26,29,29,6,5,5,5.333333,13,16,14
4,nfp8031,27,13.0,6,30,21.948577,4,28,27,27,7,5,6,6.000000,14,15,13


In [5]:
df = df.drop(['ClientID','CycleNumber','LengthofCycle1','LengthofCycle2','LengthofCycle3','LengthOfMenses1','LengthOfMenses2','LengthOfMenses3','MeanOfMenses','EstimatedDayofOvulation1','EstimatedDayofOvulation2','EstimatedDayofOvulation3'	], axis=1)

In [6]:
df.head(10)

,LengthofCycle,EstimatedDayofOvulation,LengthofMenses,Age,BMI
0,28,14.0,5,33,25.790533
1,26,15.0,5,23,24.958580
2,27,16.0,6,39,21.454890
3,25,12.0,6,23,22.462722
4,27,13.0,6,30,21.948577
5,28,14.0,5,31,22.629112
6,28,14.0,5,24,24.138503
7,33,21.0,6,27,18.775006
8,30,15.0,5,24,19.195547
9,30,16.0,5,35,29.950296


In [7]:
# Split dataset

X = df.drop(['EstimatedDayofOvulation'], axis=1)
y = df['EstimatedDayofOvulation']

In [8]:
# Membagi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [9]:
regresi = [RANSACRegressor(random_state=42),
           DecisionTreeRegressor(random_state=42),
           RandomForestRegressor(random_state=42)]

In [10]:
head = 4
for model in regresi[:head]:
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(X_test)
    predict_time = time()-start
    val_sil = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    score_val_sil = -val_sil
    print(model)
    print("\tWaktu latih: %0.3fs" % train_time)
    print("\tWaktu prediksi: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(y_test, y_pred))
    print("\tMSE:", mean_squared_error(y_test, y_pred))
    print("\tR2 score:", r2_score(y_test, y_pred))
    print('\tMSE validasi silang:', score_val_sil)
    print('\tRata-rata MSE validasi silang:', np.mean(score_val_sil))
    print()

RANSACRegressor(random_state=42)
	Waktu latih: 0.055s
	Waktu prediksi: 0.001s
	Explained variance: 0.5648055883455461
	MSE: 8.747591310440923
	R2 score: 0.3944441058844065
	MSE validasi silang: [2.27233533 3.7757202  9.86699057 9.96208757 8.12444735]
	Rata-rata MSE validasi silang: 6.800316203551878

DecisionTreeRegressor(random_state=42)
	Waktu latih: 0.004s
	Waktu prediksi: 0.001s
	Explained variance: 0.5085762633643566
	MSE: 7.383333333333334
	R2 score: 0.48888547034843466
	MSE validasi silang: [ 8.10416667 15.59375    14.5        22.15625    14.41304348]
	Rata-rata MSE validasi silang: 14.953442028985506

RandomForestRegressor(random_state=42)
	Waktu latih: 0.186s
	Waktu prediksi: 0.007s
	Explained variance: 0.5577597704022766
	MSE: 6.591055833333332
	R2 score: 0.5437312322129066
	MSE validasi silang: [ 2.86694167  7.66139063 11.47033229 16.00402604  6.36777391]
	Rata-rata MSE validasi silang: 8.874092907608695



In [11]:
parameter = {'criterion':['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
             'max_features':['sqrt', 'log2', None],
             'bootstrap':[True, False]
             }

grid = GridSearchCV(RandomForestRegressor(random_state=42), parameter)
model = grid.fit(X, y)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

{'bootstrap': True, 'criterion': 'absolute_error', 'max_features': 'sqrt'} 

RandomForestRegressor(criterion='absolute_error', max_features='sqrt',
                      random_state=42) 



In [12]:
rfr = RandomForestRegressor(criterion='absolute_error',max_features='sqrt', random_state=42)
rfr.fit(X_train, y_train)

RandomForestRegressor(criterion='absolute_error', max_features='sqrt',
                      random_state=42)

In [13]:
# model keras

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import Adam, RMSprop, SGD, Nadam, Adagrad
from keras.losses import MeanSquaredError
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(units=128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='linear'))

# Compile the model
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss=MeanSquaredError(), metrics=['mse'])


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1, validation_data=(X_test, y_test), callbacks=[early_stopping])



Epoch 1/100
8/8 [==============================] - 4s 32ms/step - loss: 145.6448 - mse: 145.6448 - val_loss: 120.7859 - val_mse: 120.7859
Epoch 2/100
8/8 [==============================] - 0s 7ms/step - loss: 88.5548 - mse: 88.5548 - val_loss: 82.0666 - val_mse: 82.0666
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: 58.7813 - mse: 58.7813 - val_loss: 56.4243 - val_mse: 56.4243
Epoch 4/100
8/8 [==============================] - 0s 7ms/step - loss: 39.2336 - mse: 39.2336 - val_loss: 39.3088 - val_mse: 39.3088
Epoch 5/100
8/8 [==============================] - 0s 7ms/step - loss: 26.7568 - mse: 26.7568 - val_loss: 27.1639 - val_mse: 27.1639
Epoch 6/100
8/8 [==============================] - 0s 7ms/step - loss: 18.8629 - mse: 18.8629 - val_loss: 19.8415 - val_mse: 19.8415
Epoch 7/100
8/8 [==============================] - 0s 7ms/step - loss: 14.6671 - mse: 14.6671 - val_loss: 15.8301 - val_mse: 15.8301
Epoch 8/100
8/8 [==============================] - 0s 7ms/step -

In [14]:
# Save the sklearn model
joblib.dump((rfr), 'prediksihariovulasi.model')
import pickle
with open('prediksihariovulasi.pkl', 'wb') as model_file:
    pickle.dump(rfr, model_file)
joblib.dump((rfr), 'prediksihariovulasi.model')


# save the kears keras
model.save('prediksihariovulasi.h5')

In [16]:
# Show the results of prediction
df_test = pd.DataFrame(data={
    'LengthofCycle': [30],
    'LengthofMenses': [5],
    'Age': [24],
    'BMI': [19],
})
pred_test = rfr.predict(df_test[0:1])
pred_test1 = model.predict(df_test[0:1])
print('Prediksi Day of Ovulation (sklearn) anda adalah ', pred_test)
print('Prediksi Day of Ovulation (keras) anda adalah', pred_test1)

1/1 [==============================] - 0s 20ms/step
Prediksi Day of Ovulation (sklearn) anda adalah  [15.445]
Prediksi Day of Ovulation (keras) anda adalah [[16.9579]]


In [17]:
# Convert the Keras model to TensorFlow.js format
!tensorflowjs_converter --input_format=keras /kaggle/working/prediksihariovulasi.h5 /kaggle/working/tfjs_model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
